In [68]:
import numpy as np
from scipy.linalg import eig

In [69]:
# define the parameters for the system

E = 0.0
t = -1.0
v = 0.0
N = 5
M = 2

# set up the lead matrix  
H = np.diag(v * np.ones(N) - E) + np.diag(t * np.ones(N - 1), k=1) + np.diag(t * np.ones(N - 1), k = -1)
I = np.diag(np.ones(N))
V = np.diag(t * np.ones(N))
Null = np.zeros((N, N))
A = np.stack((np.stack((Null, I), axis=1), np.stack((V, H), axis=1))).reshape(2*N, 2*N)
B = np.stack((np.stack((-I, Null), axis=1), np.stack((Null, V), axis=1))).reshape(2*N, 2*N)
print(A)
print(B)

[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [-1.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0. -1.  0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0. -1.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0. -1.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0. -1.  0.  0.  0. -1.  0.]]
[[-1. -0. -0. -0. -0.  0.  0.  0.  0.  0.]
 [-0. -1. -0. -0. -0.  0.  0.  0.  0.  0.]
 [-0. -0. -1. -0. -0.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -1. -0.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]]


In [70]:
# diagonalize the lead matrix
w, v = eig(A, B)
print(w)

[ 0.8660254+0.5      j  0.8660254-0.5      j  0.5      +0.8660254j
  0.5      -0.8660254j -0.8660254+0.5      j -0.8660254-0.5      j
 -0.5      +0.8660254j -0.5      -0.8660254j  0.       +1.       j
  0.       -1.       j]


In [80]:
# order the modes based on propagating, left/right moving
modes = [vec[:N] for vec in v.T]
self_energy_left = 0.
self_energy_right = 0.
V_L = np.diag(t * np.ones(N))
V_R = np.diag(t * np.ones(N))

org_modes = {'evan_R': [], 'evan_L': [], 'prop_R': [], 'prop_L': []}
for e, m in zip(w, modes):
    current = 2 * (m.conjugate().dot(V_L.dot(e * m))).imag
    print(np.abs(np.absolute(e) - 1))
    if np.abs(np.absolute(e) - 1) < 1e-8 :
        if current < 0:
            org_modes['prop_R'].append(m)
        else:
            org_modes['prop_L'].append(m)
    else:
        if current < 0:
            org_modes['evan_L'].append(m)
        else:
            org_modes['evan_R'].append(m)
    

print(org_modes)
for i, m in enumerate(modes):
    self_energy_left += w[i]**(-1) * np.outer(m, m.conjugate())
    self_energy_right += w[i] * np.outer(m, m.conjugate())
self_energy_left *= V_L
self_energy_right *= V_R

2.220446049250313e-16
0.0
4.440892098500626e-16
6.661338147750939e-16
4.440892098500626e-16
6.661338147750939e-16
5.551115123125783e-16
5.551115123125783e-16
0.0
0.0
{'evan_R': [], 'evan_L': [], 'prop_R': [array([-0.20396149-0.0081473 j, -0.35327166-0.01411154j,
       -0.40792297-0.0162946 j, -0.35327166-0.01411154j,
       -0.20396149-0.0081473 j]), array([ 3.12101759e-01-1.66109879e-01j,  3.12101759e-01-1.66109879e-01j,
        5.88287477e-17+2.96195725e-16j, -3.12101759e-01+1.66109879e-01j,
       -3.12101759e-01+1.66109879e-01j]), array([ 0.19086803+0.07236062j, -0.33059313-0.12533228j,
        0.38173607+0.14472125j, -0.33059313-0.12533228j,
        0.19086803+0.07236062j]), array([ 3.53269149e-01-1.41742136e-02j, -3.53269149e-01+1.41742136e-02j,
        1.31514292e-16-2.82415650e-16j,  3.53269149e-01-1.41742136e-02j,
       -3.53269149e-01+1.41742136e-02j]), array([-4.08248290e-01+6.62398063e-31j,  8.82471104e-32+4.44036943e-17j,
        4.08248290e-01+1.20506081e-31j,  2.451308

In [44]:
rows = [None] * M

for i in range(M):
    if i == 0:
        rows[i] = H + self_energy_left
    elif i == 1:
        rows[i] = V
    else:
        rows[i] = Null
    
for i in range(M):
    for j in range(1,M):
        if i == j:
            if j == M - 1:
                rows[i] = np.stack((rows[i], H + self_energy_right), axis=1).reshape((j + 1)*N, N)
            else:
                rows[i] = np.stack((rows[i], H), axis=1).reshape((j + 1)*N, N)
        if i == j - 1 or i == j + 1:
            rows[i] = np.stack((rows[i], V), axis=1).reshape((j + 1)*N, N)

total_H = np.stack((rows[0], rows[1])).reshape(N*M ,N*M)
psi = np.linalg.solve(total_H, np.ones(2 * N))
print(psi)

ValueError: operands could not be broadcast together with shapes (5,5) (2,2) 